In [9]:
#신영

import sys

print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


In [2]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

In [11]:
# 파이토치 설치_ conda에서 전역 설치 
# conda install pytorch torchvision torchaudio cpuonly -c pytorch

# 케라스 설치
!pip install keras

In [12]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [13]:
df=pd.read_csv("test_data1.tsv", sep="\t", header=None, encoding="UTF-8", names=['label','document',])

In [14]:
#중복 제거
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['label'] = df.label.replace(label_dict)

In [15]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = df[:1100]
test = df[1100:]

print(train.shape)
print(test.shape)

(1100, 2)
(266, 2)


In [16]:
# 훈련셋의 앞부분 출력
train.head(10)

,label,document
0,0,오예 오늘 휴강이다!!!
1,1,"사랑한다. 이 세상 하나뿐인 내 아내와 아들, 딸"
2,2,오늘 감기에 걸렸다..
3,0,놀이공원 놀러왔다 신난다! >_<
4,2,아 외롭다..ㅜ
5,3,와 진짜 억울하다 프로젝트 다 짰는데 날렸다.... 미친거 아냐?? 아...
6,0,아싸 시험 100점 ^^
7,2,오늘 키우던 개가 죽었어...
8,4,헐 벌써 4월이야? 시간이 너무 빠르다!
9,0,대박 길 걷다가 만원 주웠다 ㅋㅋㅋ


In [17]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                 오예 오늘 휴강이다!!!
1                   사랑한다. 이 세상 하나뿐인 내 아내와 아들, 딸
2                                  오늘 감기에 걸렸다..
3                            놀이공원 놀러왔다 신난다! >_<
4                                      아 외롭다..ㅜ
5    와 진짜 억울하다 프로젝트 다 짰는데 날렸다.... 미친거 아냐?? 아...
6                                 아싸 시험 100점 ^^
7                              오늘 키우던 개가 죽었어...
8                        헐 벌써 4월이야? 시간이 너무 빠르다!
9                           대박 길 걷다가 만원 주웠다 ㅋㅋㅋ
Name: document, dtype: object

In [18]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 오예 오늘 휴강이다!!! [SEP]',
 '[CLS] 사랑한다. 이 세상 하나뿐인 내 아내와 아들, 딸 [SEP]',
 '[CLS] 오늘 감기에 걸렸다.. [SEP]',
 '[CLS] 놀이공원 놀러왔다 신난다! >_< [SEP]',
 '[CLS] 아 외롭다..ㅜ [SEP]',
 '[CLS] 와 진짜 억울하다 프로젝트 다 짰는데 날렸다.... 미친거 아냐?? 아... [SEP]',
 '[CLS] 아싸 시험 100점 ^^ [SEP]',
 '[CLS] 오늘 키우던 개가 죽었어... [SEP]',
 '[CLS] 헐 벌써 4월이야? 시간이 너무 빠르다! [SEP]',
 '[CLS] 대박 길 걷다가 만원 주웠다 ㅋㅋㅋ [SEP]']

In [19]:
train['label'].value_counts()

0    498
2    233
3    120
6    115
4     79
1     37
5     17
7      1
Name: label, dtype: int64

In [20]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 1, 2, ..., 0, 0, 4], dtype=int64)

In [21]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[CLS] 오예 오늘 휴강이다!!! [SEP]
['[CLS]', '오', '##예', '오', '##늘', '휴', '##강', '##이다', '!', '!', '!', '[SEP]']


In [22]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 256

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9580,  96279,   9580, 118762,  10013,  47181,  11925,
          106,    106,    106,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [23]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [24]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=41, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=41, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9580, 118762,   8985,  49212,   9004,  32537,   9685,  11903,
           106,    106,    106,   9113,  48446,  12965,   9360,  10739,   9594,
         16439, 106154,   9389,  12692,   9343, 118691,   9946,  14867,   9685,
        118632,  11903,    100,   9429, 118873,  11903,    106,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [25]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [26]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

1100                                           수고했어, 지훈아!
1101                         행복요정이라더니..행복(부수기)요정인가...?(환장
1102         ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㅁㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이것이 흥개님의 사랑법,,♡ 
1103    까우와 까우맘이 좋아해 가끔 부산에 출장 갈 때마다한번도 잊지 않고 어묵을 산 곳인...
1104                     헐.. 메일 보내셨구나.. 메일 귀찮아서 안보냈어요 ㅋㅋㅋ
1105              ㅌㅌㅋㅋ몽이가일케기엽ㅂ습니다..자식자랑나라사랑..(저유령계정댓나바요ㅠ 
1106                   헐 당연하죠😊🙏🙏 꼬옥 남겨줄테니 다음에도 찍어주기~~~^^🤙
1107                         헐 슬로건ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㄴㅋㄴㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
1108    ㅠㅠㅠㅠㅠㅠ 아 개 넘한 ㅠㅠㅠㅠㅠㅠㅠㅠㅠ 헐 ㅠㅠㅠㅠㅠㅠ 아 진짜 넘해요,,,,,...
1109                            헉헉헉 대박!!! 박성진!! 깜짝 놀랐어😨😨 
Name: document, dtype: object

In [27]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 수고했어, 지훈아! [SEP]',
 '[CLS] 행복요정이라더니..행복(부수기)요정인가...?(환장 [SEP]',
 '[CLS] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㅁㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이것이 흥개님의 사랑법,,♡  [SEP]',
 '[CLS] 까우와 까우맘이 좋아해 가끔 부산에 출장 갈 때마다한번도 잊지 않고 어묵을 산 곳인데.. 이제는 이런 소소한 행복과 추억 마저 안녕이네... ㅜ.  [SEP]',
 '[CLS] 헐.. 메일 보내셨구나.. 메일 귀찮아서 안보냈어요 ㅋㅋㅋ [SEP]',
 '[CLS] ㅌㅌㅋㅋ몽이가일케기엽ㅂ습니다..자식자랑나라사랑..(저유령계정댓나바요ㅠ  [SEP]',
 '[CLS] 헐 당연하죠😊🙏🙏 꼬옥 남겨줄테니 다음에도 찍어주기~~~^^🤙 [SEP]',
 '[CLS] 헐 슬로건ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㄴㅋㄴㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ [SEP]',
 '[CLS] ㅠㅠㅠㅠㅠㅠ 아 개 넘한 ㅠㅠㅠㅠㅠㅠㅠㅠㅠ 헐 ㅠㅠㅠㅠㅠㅠ 아 진짜 넘해요,,,,,,ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ [SEP]',
 '[CLS] 헉헉헉 대박!!! 박성진!! 깜짝 놀랐어😨😨  [SEP]']

In [28]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 0, 2, 0, 0, 0, 0, 2, 4, 0, 2, 0, 1, 2, 2, 0, 0, 4, 0, 6, 4,
       2, 4, 3, 4, 4, 3, 2, 1, 2, 4, 1, 0, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0,
       0, 0, 2, 0, 2, 0, 3, 2, 0, 0, 0, 4, 3, 1, 2, 3, 2, 0, 6, 0, 2, 0,
       3, 6, 0, 3, 3, 2, 6, 3, 0, 2, 0, 6, 1, 2, 3, 3, 2, 3, 6, 3, 0, 2,
       3, 0, 3, 2, 3, 3, 2, 2, 2, 3, 6, 4, 0, 0, 0, 0, 2, 4, 2, 6, 1, 2,
       0, 4, 4, 4, 2, 1, 1, 6, 0, 0, 2, 1, 0, 2, 6, 4, 6, 0, 0, 6, 6, 6,
       0, 0, 0, 0, 0, 0, 0, 5, 0, 2, 4, 2, 0, 2, 1, 4, 6, 6, 4, 4, 4, 1,
       6, 1, 6, 6, 2, 0, 0, 0, 6, 0, 4, 4, 4, 4, 2, 1, 2, 0, 0, 4, 1, 2,
       4, 2, 0, 3, 0, 0, 2, 0, 3, 2, 6, 0, 2, 0, 2, 0, 0, 4, 1, 5, 6, 0,
       6, 4, 3, 3, 6, 0, 2, 0, 2, 0, 6, 0, 3, 3, 0, 3, 2, 6, 2, 0, 2, 0,
       3, 0, 6, 0, 0, 3, 2, 2, 2, 3, 0, 0, 0, 2, 6, 2, 2, 2, 2, 0, 0, 0,
       2, 3, 6, 0, 6, 0, 3, 2, 0, 3, 6, 0, 2, 0, 0, 1, 5, 3, 3, 2, 0, 4,
       6, 3], dtype=int64)

In [29]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 수고했어, 지훈아! [SEP]
['[CLS]', '수', '##고', '##했', '##어', ',', '지', '##훈', '##아', '!', '[SEP]']


In [30]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9460,  11664, 119424,  12965,    117,   9706,  75965,
        16985,    106,    102,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [31]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [32]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9460,  11664, 119424,  12965,    117,   9706,  75965,  16985,
           106,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [33]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

모델 생성

In [34]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

SystemError: GPU device not found

In [35]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [36]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_dict))
model.cuda()

# coda 없이 학습하는 방법 알아내기 과제


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

모델 학습


In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 19
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")